In [ ]:
from pathlib import Path
import sys
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "SQL").is_dir():
            return p
    return start


repo_root = find_repo_root(Path.cwd())

if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

sql_env = repo_root / "SQL" / ".env"
root_env = repo_root / ".env"
dotenv_path = sql_env if sql_env.exists() else (root_env if root_env.exists() else None)
if dotenv_path:
    load_dotenv(dotenv_path=dotenv_path)
else:
    load_dotenv()

from SQL.snowflake_client import SnowflakeClient

# Create a reusable client for the session
if 'client' in globals():
    try:
        client.close()
    except Exception:
        pass
client = SnowflakeClient.from_env()

In [ ]:
# Sample: select from calls with triple-quoted SQL
query = """
SELECT * FROM calls
LIMIT 100
"""

df = client.query_to_df(query)
df.head(100)


In [ ]:
df = client.query_to_df("select * from calls limit 10")
df.head(100)


/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,CALL_ID,CALL_START_TIME,OUTSIDE_WORKING_HOURS,DIRECTION,OUTBOUND_CALL_ANSWERED,INBOUND_CALL_ANSWERED,FIRST_AGENT_ANSWER_TIME,TOTAL_IVR,TOTAL_QUEUE_UNANSWERED,TOTAL_QUEUE_WAIT,...,COST_SUM,SLA_BREACH,ABANDON,ABANDON_WITHIN_10S,ENVIRONMENT,START_DAY,WEEK,START_HOUR,MONTH,YEAR
0,00000000-01dc-04c7-d629-df1700001aa5,2025-08-04 00:42:05,1,Inbound,0,0,None,0,32.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,0,8,2025
1,00000000-01dc-04fc-50e9-939400001aa6,2025-08-04 06:57:45,1,Inbound,0,0,None,45,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,6,8,2025
2,00000000-01dc-04fe-cac3-009300001aa7,2025-08-04 07:15:28,1,Inbound,0,0,None,36,13.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
3,00000000-01dc-04fe-f6a4-4da000001aa8,2025-08-04 07:16:42,1,Inbound,0,0,None,44,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
4,00000000-01dc-0501-6f1a-db8900001aa9,2025-08-04 07:34:23,1,Inbound,0,0,None,31,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
5,00000000-01dc-0501-900f-9cb100001aaa,2025-08-04 07:35:18,1,Inbound,0,0,None,47,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
6,00000000-01dc-0502-a26f-edd100001aab,2025-08-04 07:42:58,1,Inbound,0,0,None,14,12.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
7,00000000-01dc-0503-c383-542b00001aac,2025-08-04 07:51:03,1,Inbound,0,0,None,39,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
8,00000000-01dc-0503-d488-53f400001aad,2025-08-04 07:51:32,1,Inbound,0,0,None,24,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025
9,00000000-01dc-0504-7ff8-839200001aae,2025-08-04 07:56:20,1,Inbound,0,0,None,25,0.0,0.0,...,0.0,0,0,0,None,2025-08-04,32,7,8,2025


In [5]:
df = client.query_to_df("WITH inbound AS (
  SELECT
    START_DAY,
    START_HOUR,
    ABANDON,
    IFF(TOTAL_QUEUE_UNANSWERED IS NOT NULL OR TOTAL_QUEUE_WAIT IS NOT NULL, 1, 0) AS entered_queue
  FROM calls
  WHERE DIRECTION = 'Inbound'
)
SELECT
  START_HOUR,
  COUNT_IF(entered_queue = 1)                                        AS calls_entered_queue,
  COUNT_IF(entered_queue = 1 AND ABANDON = 1)                        AS abandons,
  ROUND(
    100 * COUNT_IF(entered_queue = 1 AND ABANDON = 1)
      / NULLIF(COUNT_IF(entered_queue = 1), 0)
  , 2)                                                               AS abandonment_rate_pct,
  COUNT(DISTINCT START_DAY)                                          AS days_observed
FROM inbound
GROUP BY START_HOUR
ORDER BY START_HOUR")
df.head(100)

SyntaxError: unterminated string literal (detected at line 1) (3744820202.py, line 1)

In [8]:
query = """
WITH inbound AS (
  SELECT
    START_DAY,
    START_HOUR,
    ABANDON,
    IFF(TOTAL_QUEUE_UNANSWERED IS NOT NULL OR TOTAL_QUEUE_WAIT IS NOT NULL, 1, 0) AS entered_queue
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND OUTSIDE_WORKING_HOURS = 0
)
SELECT
  START_HOUR,
  COUNT_IF(entered_queue = 1) AS calls_entered_queue,
  COUNT_IF(entered_queue = 1 AND ABANDON = 1) AS abandons,
  ROUND(
    100 * COUNT_IF(entered_queue = 1 AND ABANDON = 1)
      / NULLIF(COUNT_IF(entered_queue = 1), 0)
  , 2) AS abandonment_rate_pct
FROM inbound
GROUP BY START_HOUR
ORDER BY START_HOUR;
"""

df = client.query_to_df(query)
df.head(100)


/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,CALLS_ENTERED_QUEUE,ABANDONS,ABANDONMENT_RATE_PCT
0,8,216,4,1.85
1,9,355,22,6.20
2,10,301,12,3.99
3,11,313,12,3.83
4,12,355,14,3.94
5,13,301,7,2.33
6,14,265,13,4.91
7,15,316,18,5.70
8,16,298,19,6.38
9,17,224,9,4.02


In [5]:
query = """
WITH inbound AS (
  SELECT
    NVL(NULLIF(ENVIRONMENT, ''), 'Unknown') AS environment,
    START_DAY,
    ABANDON,
    IFF(TOTAL_QUEUE_UNANSWERED IS NOT NULL OR TOTAL_QUEUE_WAIT IS NOT NULL, 1, 0) AS entered_queue
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND OUTSIDE_WORKING_HOURS = 0
)
SELECT
  environment,
  COUNT_IF(entered_queue = 1)                                        AS calls_entered_queue,
  COUNT_IF(entered_queue = 1 AND ABANDON = 1)                        AS abandons,
  ROUND(
    100 * COUNT_IF(entered_queue = 1 AND ABANDON = 1)
      / NULLIF(COUNT_IF(entered_queue = 1), 0)
  , 2)                                                               AS abandonment_rate_pct,
  COUNT(DISTINCT START_DAY)                                          AS days_observed
FROM inbound
GROUP BY environment
ORDER BY abandonment_rate_pct DESC NULLS LAST, calls_entered_queue DESC;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,ENVIRONMENT,CALLS_ENTERED_QUEUE,ABANDONS,ABANDONMENT_RATE_PCT,DAYS_OBSERVED
0,Unknown,2878,126,4.38,5
1,DE/AT Wealth,111,3,2.70,5
2,DE/AT Broker,205,3,1.46,5
3,DE/AT B2B,26,0,0.00,5
4,DE/AT Fraud and 3PM,2,0,0.00,1


In [6]:
query = """
WITH answered AS (
  SELECT
    START_HOUR,
    (COALESCE(TOTAL_QUEUE_WAIT,0) + COALESCE(TOTAL_RING,0)) AS asa_sec,
    IFF(COALESCE(SLA_BREACH,0)=1, 1, 0) AS sla_breach_flag
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND INBOUND_CALL_ANSWERED = 1
    AND OUTSIDE_WORKING_HOURS = 0
)
SELECT
  START_HOUR,
  COUNT(*)                                              AS answered_calls,
  ROUND(AVG(asa_sec), 2)                                AS avg_asa_sec,
  APPROX_PERCENTILE(asa_sec, 0.5)                       AS p50_asa_sec,
  APPROX_PERCENTILE(asa_sec, 0.9)                       AS p90_asa_sec,
  ROUND(100 * AVG(sla_breach_flag), 2)                  AS sla_breach_rate_pct
FROM answered
GROUP BY START_HOUR
ORDER BY START_HOUR;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,ANSWERED_CALLS,AVG_ASA_SEC,P50_ASA_SEC,P90_ASA_SEC,SLA_BREACH_RATE_PCT
0,8,145,73.34,44.0,148.6,30.34
1,9,265,155.42,60.0,419.6,46.42
2,10,222,91.31,56.0,204.6,35.59
3,11,236,97.47,49.5,263.5,34.75
4,12,243,106.15,54.0,302.0,37.86
5,13,208,80.47,51.0,174.2,29.81
6,14,183,110.45,48.0,188.2,30.05
7,15,225,71.80,48.0,161.6,30.22
8,16,207,85.52,46.0,215.4,30.43
9,17,155,81.66,45.0,191.0,20.65


In [7]:
query = """
-- Conversion metrics by hour (in-hours, inbound only)
-- 1) Inbound Answer Rate (%)           = answered_inbound / inbound_calls
-- 2) Queued-to-Answered Ratio          = queued_calls   / answered_inbound  (lower is better)
-- 3) Queue Answer Rate (%)             = answered_from_queue / queued_calls

WITH base AS (
  SELECT
    START_HOUR,
    -- robust "entered queue" flag (use >0 to handle 0 vs NULL). If you prefer your strict NULL logic, swap line below.
    IFF(COALESCE(TOTAL_QUEUE_WAIT,0) > 0 OR COALESCE(TOTAL_QUEUE_UNANSWERED,0) > 0, 1, 0) AS entered_queue,
    IFF(INBOUND_CALL_ANSWERED = 1, 1, 0) AS answered_flag
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND OUTSIDE_WORKING_HOURS = 0
)
SELECT
  START_HOUR,
  COUNT(*)                                                     AS inbound_calls,
  SUM(answered_flag)                                           AS answered_inbound,
  SUM(entered_queue)                                           AS queued_calls,
  SUM(IFF(entered_queue = 1 AND answered_flag = 1, 1, 0))      AS answered_from_queue,

  -- (1) Inbound Answer Rate
  ROUND(100 * SUM(answered_flag) / NULLIF(COUNT(*), 0), 2)     AS inbound_answer_rate_pct,

  -- (2) Queued-to-Answered Ratio (lower is better)
  ROUND(SUM(entered_queue) / NULLIF(SUM(answered_flag), 0), 3) AS queued_to_answered_ratio,

  -- (3) Queue Answer Rate
  ROUND(100 * SUM(IFF(entered_queue = 1 AND answered_flag = 1, 1, 0))
           / NULLIF(SUM(entered_queue), 0), 2)                 AS queue_answer_rate_pct
FROM base
GROUP BY START_HOUR
ORDER BY START_HOUR;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,INBOUND_CALLS,ANSWERED_INBOUND,QUEUED_CALLS,ANSWERED_FROM_QUEUE,INBOUND_ANSWER_RATE_PCT,QUEUED_TO_ANSWERED_RATIO,QUEUE_ANSWER_RATE_PCT
0,8,216,145,140,122,67.13,0.966,87.14
1,9,355,265,247,224,74.65,0.932,90.69
2,10,301,222,212,189,73.75,0.955,89.15
3,11,313,236,216,196,75.40,0.915,90.74
4,12,355,243,231,204,68.45,0.951,88.31
5,13,301,208,183,175,69.10,0.880,95.63
6,14,265,183,171,155,69.06,0.934,90.64
7,15,316,225,204,180,71.20,0.907,88.24
8,16,298,207,190,170,69.46,0.918,89.47
9,17,224,155,132,123,69.20,0.852,93.18


In [8]:
query = """
-- Answered inbound calls by Environment (including empties as "Unattributed")
WITH answered AS (
  SELECT
    NVL(NULLIF(ENVIRONMENT, ''), 'Unattributed') AS environment
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND INBOUND_CALL_ANSWERED = 1
    AND OUTSIDE_WORKING_HOURS = 0         -- remove this line if you want all hours
)
SELECT
  environment,
  COUNT(*) AS answered_calls,
  ROUND(100 * COUNT(*) / NULLIF(SUM(COUNT(*)) OVER (), 0), 2) AS share_of_answered_pct
FROM answered
GROUP BY environment
ORDER BY answered_calls DESC, environment;;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,ENVIRONMENT,ANSWERED_CALLS,SHARE_OF_ANSWERED_PCT
0,Unattributed,1947,85.17
1,DE/AT Broker,203,8.88
2,DE/AT Wealth,109,4.77
3,DE/AT B2B,26,1.14
4,DE/AT Fraud and 3PM,1,0.04


In [ ]:
query = """
-- Conversion metrics by hour (in-hours, inbound only)
-- (1) Queue Entry Rate (%)  = queued_calls / inbound_calls
-- (2) Inbound Answer Rate (%) = answered_calls / inbound_calls
-- Last two columns show the same metrics as OVERALL totals (all in-hours).

WITH base AS (
  SELECT
    START_HOUR,
    IFF(TOTAL_QUEUE_UNANSWERED IS NOT NULL OR TOTAL_QUEUE_WAIT IS NOT NULL, 1, 0) AS entered_queue,
    IFF(INBOUND_CALL_ANSWERED = 1, 1, 0) AS answered_flag
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND OUTSIDE_WORKING_HOURS = 0
),
hour_agg AS (
  SELECT
    START_HOUR,
    COUNT(*)                               AS inbound_calls,
    SUM(entered_queue)                     AS queued_calls,
    SUM(answered_flag)                     AS answered_calls
  FROM base
  GROUP BY START_HOUR
),
totals AS (
  SELECT
    SUM(inbound_calls)  AS total_inbound_calls,
    SUM(queued_calls)   AS total_queued_calls,
    SUM(answered_calls) AS total_answered_calls
  FROM hour_agg
)
SELECT
  h.START_HOUR,
  ROUND(100 * h.queued_calls   / NULLIF(h.inbound_calls, 0), 2) AS queue_entry_rate_pct,
  ROUND(100 * h.answered_calls / NULLIF(h.inbound_calls, 0), 2) AS inbound_answer_rate_pct,
  -- Overall (all in-hours) benchmarks for comparison:
  ROUND(
    100 * (SELECT total_queued_calls FROM totals) 
        / NULLIF((SELECT total_inbound_calls FROM totals), 0), 2
  ) AS overall_queue_entry_rate_pct,
  ROUND(
    100 * (SELECT total_answered_calls FROM totals) 
        / NULLIF((SELECT total_inbound_calls FROM totals), 0), 2
  ) AS overall_inbound_answer_rate_pct
FROM hour_agg 
ORDER BY h.START_HOUR;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,QUEUE_ENTRY_RATE_PCT,INBOUND_ANSWER_RATE_PCT,OVERALL_QUEUE_ENTRY_RATE_PCT,OVERALL_INBOUND_ANSWER_RATE_PCT
0,8,100.0,67.13,100.0,70.95
1,9,100.0,74.65,100.0,70.95
2,10,100.0,73.75,100.0,70.95
3,11,100.0,75.40,100.0,70.95
4,12,100.0,68.45,100.0,70.95
5,13,100.0,69.10,100.0,70.95
6,14,100.0,69.06,100.0,70.95
7,15,100.0,71.20,100.0,70.95
8,16,100.0,69.46,100.0,70.95
9,17,100.0,69.20,100.0,70.95


In [7]:
query = """
-- In-hours inbound only
-- (1) No-Queue Rate (%) = % of inbound calls that never reached queue (and were not answered)
-- (2) No-Talk Rate (%)  = % of inbound calls that did not result in a conversation (not answered)

WITH base AS (
  SELECT
    START_HOUR,
    -- Not reached queue: both queue metrics = 0 and no answer
    IFF(
      COALESCE(TOTAL_QUEUE_WAIT,0) = 0
      AND COALESCE(TOTAL_QUEUE_UNANSWERED,0) = 0
      AND COALESCE(INBOUND_CALL_ANSWERED,0) = 0,
      1, 0
    ) AS not_reached_queue_flag,
    -- No talk: not answered
    IFF(COALESCE(INBOUND_CALL_ANSWERED,0) = 0, 1, 0) AS no_talk_flag
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND OUTSIDE_WORKING_HOURS = 0
),
hour_agg AS (
  SELECT
    START_HOUR,
    COUNT(*)                               AS inbound_calls,
    SUM(not_reached_queue_flag)            AS not_reached_queue_calls,
    SUM(no_talk_flag)                      AS no_talk_calls
  FROM base
  GROUP BY START_HOUR
),
totals AS (
  SELECT
    SUM(inbound_calls)             AS total_inbound_calls,
    SUM(not_reached_queue_calls)   AS total_not_reached_queue_calls,
    SUM(no_talk_calls)             AS total_no_talk_calls
  FROM hour_agg
)
SELECT
  h.START_HOUR,
  ROUND(100 * h.not_reached_queue_calls / NULLIF(h.inbound_calls, 0), 2) AS no_queue_rate_pct,
  ROUND(100 * h.no_talk_calls           / NULLIF(h.inbound_calls, 0), 2) AS no_talk_rate_pct,
  -- Overall benchmarks (all in-hours)
  ROUND(
    100 * (SELECT total_not_reached_queue_calls FROM totals)
        / NULLIF((SELECT total_inbound_calls FROM totals), 0), 2
  ) AS overall_no_queue_rate_pct,
  ROUND(
    100 * (SELECT total_no_talk_calls FROM totals)
        / NULLIF((SELECT total_inbound_calls FROM totals), 0), 2
  ) AS overall_no_talk_rate_pct
FROM hour_agg h
ORDER BY h.START_HOUR;;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,NO_QUEUE_RATE_PCT,NO_TALK_RATE_PCT,OVERALL_NO_QUEUE_RATE_PCT,OVERALL_NO_TALK_RATE_PCT
0,8,24.54,32.87,22.81,29.05
1,9,18.87,25.35,22.81,29.05
2,10,18.60,26.25,22.81,29.05
3,11,18.21,24.60,22.81,29.05
4,12,23.94,31.55,22.81,29.05
5,13,28.24,30.90,22.81,29.05
6,14,24.91,30.94,22.81,29.05
7,15,21.20,28.80,22.81,29.05
8,16,23.83,30.54,22.81,29.05
9,17,26.79,30.80,22.81,29.05


In [3]:
query = """
WITH answered AS (
  SELECT
    START_HOUR,
    COALESCE(TOTAL_TALK,0) AS talk_sec
  FROM calls
  WHERE OUTSIDE_WORKING_HOURS = 0
    AND (
      (DIRECTION = 'Inbound'  AND INBOUND_CALL_ANSWERED  = 1) OR
      (DIRECTION = 'Outbound' AND OUTBOUND_CALL_ANSWERED = 1)
    )
)
SELECT
  START_HOUR,
  COUNT(*)                                   AS answered_calls,
  ROUND(AVG(talk_sec), 2)                    AS avg_talk_sec,
  APPROX_PERCENTILE(talk_sec, 0.5)           AS p50_talk_sec,
  APPROX_PERCENTILE(talk_sec, 0.9)           AS p90_talk_sec
FROM answered
GROUP BY START_HOUR
ORDER BY START_HOUR;;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,ANSWERED_CALLS,AVG_TALK_SEC,P50_TALK_SEC,P90_TALK_SEC
0,8,148,353.86,322.0,692.8
1,9,269,369.96,339.0,762.4
2,10,234,421.37,365.5,862.8
3,11,243,377.43,282.0,817.4
4,12,275,352.92,275.0,804.4
5,13,229,368.70,306.0,820.6
6,14,203,355.67,300.0,755.6
7,15,244,353.20,276.0,867.7
8,16,222,382.31,322.0,817.5
9,17,165,340.90,273.0,784.4


In [8]:
query = """
WITH answered AS (
  SELECT
    NVL(NULLIF(ENVIRONMENT,''), 'Unattributed') AS environment,
    COALESCE(TOTAL_TALK,0) AS talk_sec
  FROM calls
  WHERE DIRECTION = 'Inbound'
    AND INBOUND_CALL_ANSWERED = 1
    AND OUTSIDE_WORKING_HOURS = 0
)
SELECT
  environment,
  COUNT(*)                         AS answered_calls,
  ROUND(AVG(talk_sec), 2)          AS avg_talk_sec,
  APPROX_PERCENTILE(talk_sec,0.5)  AS p50_talk_sec,
  APPROX_PERCENTILE(talk_sec,0.9)  AS p90_talk_sec
FROM answered
GROUP BY environment
ORDER BY answered_calls DESC, environment;;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,ENVIRONMENT,ANSWERED_CALLS,AVG_TALK_SEC,P50_TALK_SEC,P90_TALK_SEC
0,Unattributed,1947,374.52,316.375,810.8
1,DE/AT Broker,203,407.50,375.000,823.0
2,DE/AT Wealth,109,331.31,252.000,726.2
3,DE/AT B2B,26,205.19,174.500,334.5
4,DE/AT Fraud and 3PM,1,0.00,0.000,0.0


In [9]:
query = """
WITH outbound AS (
  SELECT
    START_HOUR,
    IFF(OUTBOUND_CALL_ANSWERED = 1, 1, 0) AS answered_flag
  FROM calls
  WHERE DIRECTION = 'Outbound'
    AND OUTSIDE_WORKING_HOURS = 0
)
SELECT
  START_HOUR,
  COUNT(*)                                   AS outbound_calls,
  SUM(answered_flag)                         AS answered_calls,
  ROUND(100 * SUM(answered_flag) / NULLIF(COUNT(*),0), 2) AS outbound_answer_rate_pct
FROM outbound
GROUP BY START_HOUR
ORDER BY START_HOUR;;
"""

df = client.query_to_df(query)
df.head(100)

/Users/evgeniiliashko/repos/scalable_capital/SQL/snowflake_client.py:85: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(sql=sql, con=self.connection, params=params)


,START_HOUR,OUTBOUND_CALLS,ANSWERED_CALLS,OUTBOUND_ANSWER_RATE_PCT
0,8,3,3,100.00
1,9,6,4,66.67
2,10,15,12,80.00
3,11,8,7,87.50
4,12,35,32,91.43
5,13,27,21,77.78
6,14,28,20,71.43
7,15,26,19,73.08
8,16,24,15,62.50
9,17,11,10,90.91
